In [1]:
# Data driven farmer goes to the Rodeo

import numpy as np
import pylab as pl
from sklearn import svm
from sklearn import linear_model
from sklearn import tree
import pandas as pd


In [2]:
def plot_results_with_hyperplane(clf, clf_name, df, plt_nmbr):
    x_min, x_max = df.x.min() - .5, df.x.max() + .5
    y_min, y_max = df.y.min() - .5, df.y.max() + .5

    # step between points. i.e. [0, 0.02, 0.04, ...]
    step = .02
    # to plot the boundary, we're going to create a matrix of every possible point
    # then label each point as a wolf or cow using our classifier
    xx, yy = np.meshgrid(np.arange(x_min, x_max, step), np.arange(y_min, y_max, step))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    # this gets our predictions back into a matrix
    Z = Z.reshape(xx.shape)

    # create a subplot (we're going to have more than 1 plot on a given image)
    pl.subplot(2, 2, plt_nmbr)
    # plot the boundaries
    pl.pcolormesh(xx, yy, Z, cmap=pl.cm.Paired)

    # plot the wolves and cows
    for animal in df.animal.unique():
        pl.scatter(df[df.animal==animal].x,
                   df[df.animal==animal].y,
                   marker=animal,
                   label="cows" if animal=="x" else "wolves",
                   color='black')
    pl.title(clf_name)
    pl.legend(loc="best")


In [18]:
# Source: https://gist.githubusercontent.com/glamp/4365660/raw/474658c2c5a8bb763c50278c810d45a27bd21c7e/cows_and_wolves.txt
import urllib2
target_url = "https://gist.githubusercontent.com/glamp/4365660/raw/474658c2c5a8bb763c50278c810d45a27bd21c7e/cows_and_wolves.txt"
data = urllib2.urlopen(target_url)
temp = ""
for line in data: # files are iterable
    temp = temp + line
#data = open("cows_and_wolves.txt").read()
# print (temp)
data = [row.split('\t') for row in temp.split('\n')]
# print data.shape
print (data)

[['  o', '', 'o', '', 'o', ''], ['', 'o', '', ' ', 'x', '', ''], ['o', '', '', 'x', 'x', 'x', ''], ['', '', '', 'x', '', '', 'o'], ['', '', '', 'x', 'x', 'x', 'o'], ['', '', 'x', '', '', '', ''], ['', '', '', '', 'o', '', 'o'], ['', '', 'o', '', '', '', ''], ['', '', '', '', '', '', '']]


In [19]:
animals = []
for y, row in enumerate(data):
    for x, item in enumerate(row):
        # x's are cows, o's are wolves
        if item in ['o', 'x']:
            animals.append([x, y, item])

In [22]:
animals

[[2, 0, 'o'],
 [4, 0, 'o'],
 [1, 1, 'o'],
 [4, 1, 'x'],
 [0, 2, 'o'],
 [3, 2, 'x'],
 [4, 2, 'x'],
 [5, 2, 'x'],
 [3, 3, 'x'],
 [6, 3, 'o'],
 [3, 4, 'x'],
 [4, 4, 'x'],
 [5, 4, 'x'],
 [6, 4, 'o'],
 [2, 5, 'x'],
 [4, 6, 'o'],
 [6, 6, 'o'],
 [2, 7, 'o']]

In [23]:
df = pd.DataFrame(animals, columns=["x", "y", "animal"])
df['animal_type'] = df.animal.apply(lambda x: 0 if x=="x" else 1)


In [24]:
df

,x,y,animal,animal_type
0,2,0,o,1
1,4,0,o,1
2,1,1,o,1
3,4,1,x,0
4,0,2,o,1
5,3,2,x,0
6,4,2,x,0
7,5,2,x,0
8,3,3,x,0
9,6,3,o,1


In [25]:
# train using the x and y position coordiantes
train_cols = ["x", "y"]


In [26]:
clfs = {
    "SVM": svm.SVC(),
    "Logistic" : linear_model.LogisticRegression(),
    "Decision Tree": tree.DecisionTreeClassifier(),
}

In [27]:
plt_nmbr = 1
for clf_name, clf in clfs.iteritems():
    clf.fit(df[train_cols], df.animal_type)
    plot_results_with_hyperplane(clf, clf_name, df, plt_nmbr)
    plt_nmbr += 1
pl.show()

/System/Library/Frameworks/Python.framework/Versions/2.7/Extras/lib/python/matplotlib/collections.py:548: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == 'face':
